In [113]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from tqdm import tqdm

#### Hyperparameters

In [115]:
RANDOM_STATE = 42
BATCH_SIZE = 32
NUM_EPOCHS = 40
LEARNING_RATE = 1e-3
PATIENCE = 10
MIN_DELTA = 1e-4

## Loading Data

In [70]:
processed_global_run_data = pd.read_parquet('processed_global_run_data.parquet')
processed_global_incoming_run_data = pd.read_parquet('processed_global_incoming_run_data.parquet')
processed_global_metrology_data = pd.read_parquet('processed_global_metrology_data.parquet')

In [3]:
processed_global_run_data.head()

,Tool ID,Run Start Time,Run ID,Consumable Life,Step ID,Time Stamp,Sensor Name,Sensor Value
0,8,0,1644,370.7229,0,0,0,-0.061105
1,8,0,1644,370.7229,0,1,0,-0.040894
2,8,0,1644,370.7229,0,2,0,0.005357
3,8,0,1644,370.7229,1,3,0,-0.029941
4,8,0,1644,370.7229,1,4,0,-0.098098


In [6]:
processed_global_run_data.columns

Index(['Tool ID', 'Run Start Time', 'Run ID', 'Consumable Life', 'Step ID',
       'Time Stamp', 'Sensor Name', 'Sensor Value'],
      dtype='object')

In [64]:
run_0 = processed_global_run_data.groupby('Run ID').get_group(0)
run_0_data = run_0.pivot(index='Time Stamp', columns='Sensor Name', values='Sensor Value').reset_index(drop=False)
run_0_data['Run Start Time'] = run_0['Run Start Time'].iloc[0]
run_0_data['Consumable Life'] = run_0['Consumable Life'].iloc[0]
run_0_data['Step ID'] = run_0['Step ID'].iloc[0]
run_0_data['Tool ID'] = run_0['Tool ID'].iloc[0] # TODO: one-hot encode Tool ID
cols = run_0_data.columns.tolist()
cols.insert(0, cols.pop(cols.index('Run Start Time')))
cols.insert(0, cols.pop(cols.index('Step ID')))
cols.insert(0, cols.pop(cols.index('Tool ID')))
run_0_data = run_0_data[cols]
run_0_data = run_0_data.reindex(range(755), fill_value=0.0)
print(run_0_data.shape)
print(run_0_data.dtypes)
(run_0_data)

(755, 20)
Sensor Name
Tool ID            float64
Step ID            float64
Run Start Time     float64
Time Stamp         float64
0                  float32
1                  float32
2                  float32
3                  float32
4                  float32
5                  float32
6                  float32
7                  float32
8                  float32
9                  float32
10                 float32
11                 float32
12                 float32
13                 float32
14                 float32
Consumable Life    float32
dtype: object


Sensor Name,Tool ID,Step ID,Run Start Time,Time Stamp,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Consumable Life
0,18.0,0.0,67655.0,67655.0,0.031361,-685.860962,43.201454,-0.062989,0.096825,199.516937,19.552303,321.102417,302.841675,349.080383,0.067108,0.046422,713.761169,747.554504,769.712463,270.084137
1,18.0,0.0,67655.0,67656.0,-0.012847,1033.420654,49.187046,0.047596,0.090925,198.919159,20.790668,349.450714,330.221710,379.608734,0.109856,-0.056719,776.895081,814.807068,837.206055,270.084137
2,18.0,0.0,67655.0,67657.0,0.009782,55674.625000,98.741898,0.001774,0.084763,199.439133,31.275209,377.232452,356.137665,411.858246,0.077752,-0.028565,838.632385,879.301880,907.878174,270.084137
3,18.0,0.0,67655.0,67658.0,0.030168,-9624.925781,98.783913,-0.010263,0.102715,198.240219,32.151119,378.474884,356.320068,410.731934,0.111437,-0.003759,841.154785,879.696106,905.663452,270.084137
4,18.0,0.0,67655.0,67659.0,0.020838,2319.374023,98.922531,0.042651,0.069815,198.713242,32.410534,377.695648,356.672089,410.337097,0.112087,-0.012833,839.642944,880.184509,904.920410,270.084137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
751,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
752,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
753,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Data Preparation

### Process `run_data`

In [65]:
# train_run_data = processed_global_run_data.groupby('Run ID').get_group(1644).pivot(index='Time Stamp', columns='Sensor Name', values='Sensor Value')

run_matrices = []
for key, run in processed_global_run_data.groupby('Run ID'):
    run_matrix = run.pivot(index='Time Stamp', columns='Sensor Name', values='Sensor Value').reset_index(drop=False)
    run_matrix['Run Start Time'] = run['Run Start Time'].iloc[0]
    run_matrix['Consumable Life'] = run['Consumable Life'].iloc[0]
    run_matrix['Step ID'] = run['Step ID'].iloc[0]
    run_matrix['Tool ID'] = run['Tool ID'].iloc[0] # TODO: one-hot encode Tool ID

    cols = run_matrix.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Run Start Time')))
    cols.insert(0, cols.pop(cols.index('Step ID')))
    cols.insert(0, cols.pop(cols.index('Tool ID')))
    run_matrix = run_matrix[cols]

    run_matrix = run_matrix.reindex(range(755), fill_value=0.0).to_numpy()
    run_matrices.append(run_matrix)

run_matrices = np.stack(run_matrices)
print(run_matrices.shape)

(4140, 755, 20)


In [66]:
print(run_matrices[0, 0, :])

[ 1.80000000e+01  0.00000000e+00  6.76550000e+04  6.76550000e+04
  3.13611105e-02 -6.85860962e+02  4.32014542e+01 -6.29886463e-02
  9.68251526e-02  1.99516937e+02  1.95523033e+01  3.21102417e+02
  3.02841675e+02  3.49080383e+02  6.71078935e-02  4.64224853e-02
  7.13761169e+02  7.47554504e+02  7.69712463e+02  2.70084137e+02]


### Process `incoming_run_data`

In [4]:
processed_global_incoming_run_data.head()

,Tool ID,Run Start Time,Run ID,Step ID,Time Stamp,Sensor Name,Sensor Value
0,8,0,1644,0,0,0,202.660553
1,8,0,1644,0,1,0,202.660553
2,8,0,1644,0,2,0,202.660553
3,8,0,1644,1,3,0,202.660553
4,8,0,1644,1,4,0,202.660553


In [67]:
incoming_run_matrices = []
for key, incoming_run in processed_global_incoming_run_data.groupby('Run ID'):
    incoming_run_matrix = incoming_run.pivot(index='Time Stamp', columns='Sensor Name', values='Sensor Value').reset_index(drop=False)
    incoming_run_matrix['Run Start Time'] = incoming_run['Run Start Time'].iloc[0]
    incoming_run_matrix['Step ID'] = incoming_run['Step ID'].iloc[0]
    incoming_run_matrix['Tool ID'] = incoming_run['Tool ID'].iloc[0] # TODO: try deleting/one-hot encoding Tool ID

    cols = incoming_run_matrix.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Run Start Time')))
    cols.insert(0, cols.pop(cols.index('Step ID')))
    cols.insert(0, cols.pop(cols.index('Tool ID')))
    incoming_run_matrix = incoming_run_matrix[cols]

    incoming_run_matrix = incoming_run_matrix.reindex(range(755), fill_value=0.0).to_numpy()
    incoming_run_matrices.append(incoming_run_matrix)

incoming_run_matrices = np.stack(incoming_run_matrices)
print(incoming_run_matrices.shape)

(4140, 755, 45)


In [68]:
print(incoming_run_matrices[0, 0, :])

[ 1.80000000e+01  0.00000000e+00  8.25640000e+04  8.25640000e+04
  2.06448059e+02 -3.86409616e+00 -8.01727116e-01  4.86098442e+01
  9.49880886e+00  4.20048237e-01  4.50297070e+00  8.82590088e+02
  1.68778553e+01  9.57505417e+01 -1.31489173e-01  7.83298828e+02
  2.07790674e+03  4.32936859e+00  3.12357845e+01  1.84935880e+00
  1.55615845e+02  2.59801426e+01  5.00202141e+01 -5.73219657e-01
  9.07108917e+01  2.46390629e+01  1.94097729e+01  4.68600243e-02
  3.00300079e+02  7.86665466e+02  2.57429600e+00  5.47670484e-01
  7.77703629e+01  5.11192902e+02  2.00354140e-02  3.20952988e+01
  5.53313637e+01  1.02753760e+03  2.93487310e+00  5.02308884e+01
 -3.31317842e-01  2.47388439e+01  2.89062476e+00  3.37605476e+01
  5.83634224e+01]


### Process `metrology_data`

In [71]:
processed_global_metrology_data.head()

,Run ID,Point Index,Measurement
0,8,3,10.006534
1,8,48,10.002181
2,8,43,10.031223
3,8,20,10.055888
4,8,8,10.089505


In [76]:
metrology_matrix = processed_global_metrology_data.pivot(index='Run ID', columns='Point Index', values='Measurement')
metrology_matrix = metrology_matrix.to_numpy()
print(metrology_matrix.shape)

(4140, 49)


## Models

### Baseline-1: LSTM + ff

In [ ]:
class DualLSTMModel(nn.Module):
    def __init__(self,
                 run_size = 20,
                 incoming_run_size = 45,
                 run_hidden_size = 128,
                 incoming_run_hidden_size = 128,
                 num_layers = 1,
                 dropout = 0.2,
                 ff_hidden_sizes=None,
                 ff_output_size=49):
        super().__init__()
        self.run_size = run_size
        self.incoming_run_size = incoming_run_size
        self.run_hidden_size = run_hidden_size
        self.incoming_run_hidden_size = incoming_run_hidden_size

        if ff_hidden_sizes is None:
            ff_hidden_sizes = [256, 128]
        self.lstm_run = nn.LSTM(
            input_size=run_size,
            hidden_size=run_hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True,
        )

        self.lstm_incoming_run = nn.LSTM(
            input_size=incoming_run_size,
            hidden_size=incoming_run_hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
        )

        last_output_size = run_hidden_size + incoming_run_hidden_size
        ff_layers = []
        prev_hidden_size = last_output_size

        for hidden_size in ff_hidden_sizes:
            ff_layers.extend([
                nn.Linear(prev_hidden_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_hidden_size = hidden_size

        ff_layers.append(nn.Linear(prev_hidden_size, ff_output_size))
        self.fead_forward = nn.Sequential(*ff_layers)

    def forward(self, x1, x2, lengths1, lengths2):
        if lengths1 is not None:
            x1_packed = nn.utils.rnn.pack_padded_sequence(
                x1, lengths1.cpu(), batch_first=True, enforce_sorted=False
            )
            lstm1_out_packed, (h1_n, c1_n) = self.lstm_run(x1_packed)
            out_run = h1_n[-1]
        else:
            lstm1_out, (h1_n, c1_n) = self.lstm_run(x1)
            out_run = h1_n[-1]

        if lengths2 is not None:
            x2_packed = nn.utils.rnn.pack_padded_sequence(
                x2, lengths2.cpu(), batch_first=True, enforce_sorted=False
            )
            lstm2_out_packed, (h2_n, c2_n) = self.lstm_incoming_run(x2_packed)
            out_incoming_run = h2_n[-1]
        else:
            lstm2_out, (h2_n, c2_n) = self.lstm_incoming_run(x2)
            out_incoming_run = h2_n[-1]

        return self.fead_forward(torch.concat([out_run, out_incoming_run], dim=1))

In [139]:
# Model summary
from torchinfo import summary

model = DualLSTMModel()

summary(
    model,
    input_data=(
        torch.randn(32, 755, 20),  # x1: batch_size=8, seq_len=10, feature_dim=20
        torch.randn(32, 755, 45),  # x2
        torch.full((32,), 700),    # lengths1
        torch.full((32,), 700)     # lengths2
    )
)

Layer (type:depth-idx)                   Output Shape              Param #
DualLSTMModel                            [32, 49]                  --
├─LSTM: 1-1                              [22400, 128]              76,800
├─LSTM: 1-2                              [22400, 128]              89,600
├─Sequential: 1-3                        [32, 49]                  --
│    └─Linear: 2-1                       [32, 256]                 65,792
│    └─ReLU: 2-2                         [32, 256]                 --
│    └─Dropout: 2-3                      [32, 256]                 --
│    └─Linear: 2-4                       [32, 128]                 32,896
│    └─ReLU: 2-5                         [32, 128]                 --
│    └─Dropout: 2-6                      [32, 128]                 --
│    └─Linear: 2-7                       [32, 49]                  6,321
Total params: 271,409
Trainable params: 271,409
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 477.11
Input size (MB): 6.28


In [127]:
def get_sequence_lengths(x, padding_value=0.0):
    mask = (x != padding_value).any(dim=-1)
    reversed_mask = torch.flip(mask, dims=[1])
    lengths = mask.size(1) - reversed_mask.int().argmax(dim=1)
    return lengths.to(dtype=torch.long, device=x.device)

class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, x1, x2, y, padding_value=0.0):
        """
        Args:
            x1: (4140, 755, 20) tensor
            x2: (4140, 755, 45) tensor
            y: (4140, 49) tensor
            padding_value: Value used for padding (default: 0.0)
        """
        self.x1 = x1
        self.x2 = x2
        self.y = y
        self.padding_value = padding_value
        assert len(x1) == len(x2) == len(y), "All inputs must have the same number of samples!"
        self.lengths1 = get_sequence_lengths(x1, padding_value)
        self.lengths2 = get_sequence_lengths(x2, padding_value)

    def __len__(self):
        return len(self.x1)

    def __getitem__(self, item):
        return (
            self.x1[item],
            self.x2[item],
            self.y[item],
            self.lengths1[item],
            self.lengths2[item]
        )

def create_data_loaders(x1, x2, y, train_ratio=0.8, val_ratio=0.1, batch_size = BATCH_SIZE, shuffle=False, num_workers=0, padding_value=0.0, random_state=RANDOM_STATE):
    dataset = TimeSeriesDataset(x1, x2, y, padding_value)

    total_size = len(dataset)
    train_size = int(train_ratio * total_size)
    val_size = int(val_ratio * total_size)
    test_size = total_size - train_size - val_size

    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(random_state)
    )

    loader_kwargs = dict(batch_size=batch_size, num_workers=num_workers, pin_memory=True) # TODO: Try pin_memory=False

    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=shuffle, **loader_kwargs)
    val_loader = torch.utils.data.DataLoader(val_dataset, shuffle=False, **loader_kwargs)
    test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, **loader_kwargs)

    return train_loader, val_loader, test_loader

def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    num_batches = 0

    for batch_data in train_loader:
        x1, x2, y, lengths1, lengths2 = batch_data
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        lengths1, lengths2 = lengths1.to(device), lengths2.to(device)

        optimizer.zero_grad()
        outputs = model(x1, x2, lengths1, lengths2)
        loss = criterion(outputs, y)
        loss.backward()

        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # TODO: See if need this

        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

    return total_loss / num_batches

def validate_epoch(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for batch_data in val_loader:
            x1, x2, y, lengths1, lengths2 = batch_data
            x1, x2, y = x1.to(device), x2.to(device), y.to(device)
            lengths1, lengths2 = lengths1.to(device), lengths2.to(device)

            outputs = model(x1, x2, lengths1, lengths2)
            loss = criterion(outputs, y)
            total_loss += loss.item()
            num_batches += 1

    return total_loss / num_batches

def train_model(model, train_loader, val_loader, num_epochs=NUM_EPOCHS, learning_rate=LEARNING_RATE, device='cuda' if torch.cuda.is_available() else 'cpu', patience=PATIENCE, min_delta=MIN_DELTA):
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # TODO: Try AdamW, try weight_decay=1e-5
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    ) # TODO: Try different hyperparameters

    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss = validate_epoch(model, val_loader, criterion, device)
        lr_scheduler.step(val_loss)

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        # Printing
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.2e}")

        # Early stopping
        if val_loss < best_val_loss - min_delta:
            best_val_loss = val_loss
            patience_counter = 0
            # Save best model weights
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}.")
            break

        print("-" * 20)

    model.load_state_dict(torch.load('best_model.pth'))

    return train_losses, val_losses

def test_model(model, test_loader, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.eval()

    all_predictions = []
    all_targets = []
    all_losses = []

    criterion = nn.MSELoss()

    print(f"Testing model on {len(test_loader)} batches...")
    with torch.no_grad():
        for batch_data in test_loader:
            x1, x2, y, lengths1, lengths2 = batch_data
            x1, x2, y = x1.to(device), x2.to(device), y.to(device)
            lengths1, lengths2 = lengths1.to(device), lengths2.to(device)

            outputs = model(x1, x2, lengths1, lengths2)
            loss = criterion(outputs, y)
            all_losses.append(loss.item())
            all_predictions.append(outputs.cpu())
            all_targets.append(y.cpu())

    test_loss = sum(all_losses) / len(all_losses)

    all_predictions = torch.cat(all_predictions, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    results = {
        'test_loss': test_loss,
        'predictions': all_predictions,
        'targets': all_targets
    }

    # Regression metrics
    mse = torch.nn.functional.mse_loss(all_predictions, all_targets).item()
    mae = torch.nn.functional.l1_loss(all_predictions, all_targets).item()

    # R^2 score (coefficient of determination)
    ss_res = torch.sum((all_targets - all_predictions) ** 2)
    ss_tot = torch.sum((all_targets - torch.mean(all_targets)) ** 2)
    r2_score = 1 - (ss_res / ss_tot)

    rmse = torch.sqrt(torch.tensor(mse)).item()

    results.update({
        'mse': mse,
        'mae': mae,
        'rmse': rmse,
        'r2_score': r2_score.item()
    })

    return results

In [131]:
X_run = torch.from_numpy(run_matrices).float()
X_incoming_run = torch.from_numpy(incoming_run_matrices).float()
y = torch.from_numpy(metrology_matrix).float()
print(X_run.shape, X_incoming_run.shape, y.shape)

torch.Size([4140, 755, 20]) torch.Size([4140, 755, 45]) torch.Size([4140, 49])


In [133]:
baseline_1_model = DualLSTMModel(
    run_hidden_size=128,
    incoming_run_hidden_size=128,
    num_layers=1,
    dropout=0.2,
    ff_hidden_sizes=[256, 128]
)

train_loader, val_loader, test_loader = create_data_loaders(X_run, X_incoming_run, y, train_ratio=0.7, val_ratio=0.1)

train_losses, val_losses = train_model(baseline_1_model, train_loader, val_loader, num_epochs=1)

Epoch 1/1
Train Loss: 20.260741, Val Loss: 0.075996
Learning Rate: 1.00e-03
--------------------


In [134]:
test_results = test_model(baseline_1_model, test_loader)

Testing model on 26 batches...


In [137]:
print({k: test_results[k] for k in ['test_loss', 'mse', 'mae', 'r2_score']})

{'test_loss': 0.12389485280101116, 'mse': 0.12407800555229187, 'mae': 0.22869639098644257, 'r2_score': -3.1123900413513184}
